최신 체인 구성방법 V0.3
 
LLMChain, SequentialChain 등과 같이 클래스 의존도를 높임   
Runnable 공동인터페이스를 통해 일관성 유지  
핵심 : Runnable + Composition --> 프롬프트 | 모델 | 파서



In [1]:
# %pip install langchain openai python -dotenv


In [2]:
from dotenv import load_dotenv
load_dotenv()  # .env 파일의 내용을 읽어서 환경변수에 등록
import os
openai_key = os.getenv("OPENAI_API_KEY")
print(openai_key[:5]+'*****')

sk-pr*****


단일 체인 : prompt -> llm -> 출력파서 (상품설명)

In [3]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 1. 프롬프트 템플릿 정의
product_prompt = PromptTemplate.from_template(
    "제품 이름 : {product}\n"
    "이 제품의 특징과 장점을 매력적인 한 단락으로 설명해 주세요"
)

# 2. 출력파서 정의
output_parser = StrOutputParser()

# 3. llm정의
llm = ChatOpenAI(model = 'gpt-4o-mini', temperature=0)

# 4. LCEL 체인 구성 프롬프트 -> 모델 -> 출력 파서

# LCEL 출력
summary_chain = product_prompt | llm | output_parser

# invoke 메서드로 프롬프트 실행 
result = summary_chain.invoke({'product' : '선풍기'})
# 결과 출력
result

'선풍기는 더운 여름철의 필수 아이템으로, 시원한 바람을 통해 쾌적한 실내 환경을 제공합니다. 최신 기술이 적용된 이 선풍기는 조용한 작동과 다양한 풍속 조절 기능을 갖추어, 사용자의 취향에 맞춰 최적의 바람을 만들어냅니다. 또한, 슬림한 디자인과 경량 구조로 공간을 효율적으로 활용할 수 있으며, 에너지 절약 기능이 탑재되어 있어 환경을 생각하는 소비자에게도 안성맞춤입니다. 언제 어디서나 손쉽게 이동할 수 있는 이 선풍기는 여름의 더위를 잊게 해줄 완벽한 동반자가 될 것입니다.'

In [4]:
# OpenAI는 Text completion 문장을 이어서 완성하는 모델
# Text-davinci 모델을 위한 라이브러리 이전방식이라서, 역할기반 템풀릿이 없고 단순한 prompt

# from langchain_openai import OpenAI
# llm = OpenAI(model='gpt-4o-mini', temperature=0.5)
# print(llm.invoke("다음 문장을 완성하세요: 옛날옛적에 호랑이가 살았습니다."))

다중 체인 : 체인 합성 및 Runnable 병합(이메일 작성)
- 둘 이상의 llm 호출을 연결해서 복잡한 작업을 수행
- 제목 : "주어진 상황에 대한 이메일 작성"  --> 제목
- 제목을 활용해서 이메일 본문 작성 --> 본문
- Chain composition 

In [5]:
# 프롬프트 1 : 사용자로부터 받은 이메을 요청내용을 입력받아서 "이메일 제목"을 한문장으로 생성하는 명령
# llm 호출 -> 이메일 제목출력('프로젝트 진행상황 회의 일정 안내')
# 중간 출력 변환 : 생성된 제목문자열을 {subject} 키를 갖는 dictionary 변환
# 프롬프트 2 : {subject}변수를 받아서 해당 제목을 가진 이메일 본문내용을 요청
# llm 호출

In [6]:
from langchain_core.runnables import RunnablePassthrough
# 1. 이메일 생성용 프롬프트 정의
subject_prompt = PromptTemplate.from_template(
    '다음 요청 내용을 바탕으로 이메일 제목을 만들어주세요\n'
    '{content}'
)
# 2. 이메일 본문 생성용 프롬프트
body_prompt = PromptTemplate.from_template(
    '위에서 생성된 제목을 활용해서 팀에게 보내는 정중한 이메일 제목과 본문을 작성해 주세요\n'    
    '제목:{subject}\n'
    '본문:'
)
# 3. 두 프롬프트를 결합한 체인 구성
email_chain = (
    subject_prompt
    | llm
    | {'subject' : RunnablePassthrough()}  # 출력된 제목을 subjet 키로 매핑
    | body_prompt
    | llm
    | StrOutputParser()
)
# 4. 실행
result =  email_chain.invoke({
    'content' : '''대선이 1주일 앞으로 다가온 가운데 후보들은 오늘(27일) 마지막 TV 토론에 나섭니다.
                이번엔 '정치'를 주제로 치열한 공방을 벌일 예정입니다.'''
})
print(result)

제목: 대선 1주일 앞두고, 후보들 마지막 TV 토론에서 정치 공방 펼친다!

안녕하세요 팀 여러분,

다가오는 대선이 일주일 앞으로 다가왔습니다. 이에 따라 후보자들이 마지막 TV 토론에서 치열한 정치 공방을 펼칠 예정입니다. 이번 토론은 유권자들에게 중요한 결정의 기초가 될 수 있는 만큼, 우리 팀의 분석과 의견이 더욱 중요해질 것입니다.

각자 맡은 바 역할에 최선을 다해 주시고, 토론 후에는 함께 논의하여 우리의 인사이트를 공유하는 시간을 가지면 좋겠습니다. 여러분의 소중한 의견을 기다리겠습니다.

감사합니다.

[당신의 이름]  
[당신의 직책]  
[회사/팀 이름]  


# 조건분기 : 입력조건에 따라 요약 또는 이메일 작성

In [8]:
from langchain_core.runnables import RunnableBranch, RunnableLambda
# 1. 요약 체인 정의(prompt -> llm)
summary_chain =  PromptTemplate.from_template(
        "다음 문장을 한 무단으로 간결하게 요약해주세요\n"
        "{text}"
    )
summary_chain = summary_chain | llm | StrOutputParser()
# 2. email chain은 정의
# 3. 분기조건 함수 정의 => Runnable로 래핑
def is_summary_request(user_input : str) -> bool:
    return user_input.strip().startswith('요약:')
condition = RunnableLambda(is_summary_request)
# 분기 체인
branch_chain = RunnableBranch(
    (condition, summary_chain),  # 조건이 참일 때 실행할 체인
    email_chain  # 조건이 거짓일 때 실행할 체인
)
# 다양한 조건
input1 = '요약 : 오늘 회의에서는 다양한 주제에 대한 토론이 있었는데, 특히 일정 변경과 예산 관련 내용이 다수였습니다.'
input2 = '이메일 : 다음주 월요일 프로젝트 회의 일정을 팀에게 공지해줘'

result1 = branch_chain.invoke(input1)
result2 = branch_chain.invoke(input2)
print(f'result1: {result1}')
print(f'result2: {result2}')

result1: 제목: 오늘 회의 요약: 일정 변경 및 예산 관련 논의

안녕하세요, 팀 여러분.

오늘 회의에서 논의된 주요 사항들을 요약하여 공유드립니다.

1. **일정 변경**: 프로젝트 일정에 대한 변경 사항이 논의되었습니다. 새로운 일정은 다음 주 월요일부터 적용될 예정입니다. 각 팀원들은 변경된 일정을 반영하여 작업을 진행해 주시기 바랍니다.

2. **예산 관련 논의**: 예산 사용에 대한 현황과 향후 계획에 대해 논의하였습니다. 추가적인 예산 요청이 필요한 경우, 다음 회의 전까지 저에게 알려주시기 바랍니다.

회의에 참석해 주신 모든 분들께 감사드리며, 추가적인 질문이나 의견이 있으시면 언제든지 말씀해 주세요.

감사합니다.

[당신의 이름]  
[당신의 직책]  
[회사명]  
[연락처]  
result2: 제목: 다음주 월요일 프로젝트 회의 일정 공지

안녕하세요 팀원 여러분,

다음주 월요일에 예정된 프로젝트 회의 일정을 안내드립니다. 회의는 오전 10시에 시작되며, 회의실 A에서 진행될 예정입니다. 이번 회의에서는 프로젝트 진행 상황과 향후 계획에 대해 논의할 예정이니, 모두 참석해 주시기 바랍니다.

회의에 필요한 자료가 있으신 분들은 미리 준비해 주시면 감사하겠습니다. 추가로 논의하고 싶은 사항이 있으시면 회의 전에 저에게 알려주세요.

감사합니다.

좋은 하루 되세요!

[당신의 이름]  
[당신의 직책]  
[회사 이름]  
[연락처]  
